# IEOR 4501 Final Project: Understanding Hired Rides in NYC

#### Contributors: Joy Ren(jr4154), Yiwen Qian(yq2346)

## Project Setup

In [16]:
# all import statements needed for the project

import math
from math import sin, cos, sqrt, atan2, radians
import os

import re
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd


In [15]:
# any constants need; some have been added for you, and 
# some you need to fill in

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
UBER_DATA = "uber_rides_sample.csv"
WEATHER_CSV_DIR = "weather"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing
In Part 1, we downloaded Parquet files, cleaning and filtering for the relevant data, filling in missing data, and generating samples of these raw datasets.
Our processing stages can be illustrated as:
* 1. Load the Taxi Zones by x and y coordinates.
* 2. Calculate distance using latitude and longtitude.
* 3. Use `bs4` and `requests` module to parse html, then we can process Yellow Taxi data of NYC.
* 4. Processing Uber Data.
* 5. Processing Weather Data.
* 6. Save the cleaned data for further analyis. 

### Load Taxi Zones

In [4]:
def load_taxi_zones(shapefile):
    return gpd.read_file(shapefile)     #read shapefile

In [5]:
def lookup_coords_for_taxi_zone_id(zone_loc_id):
    zones= load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    centroid= zones[zones['LocationID'] == zone_loc_id].centroid.values[0]
    return (centroid.y, centroid.x)        # return the centroid coordinates

### Calculate distance

In [18]:
def calculate_distance_with_coords(from_coord, to_coord):
    ## "from_coord": columns 'pickup_latitude' and 'pickup_longitude' from datarame;
    ## "to_coord":  columns 'dropoff_latitude' and 'dropoff_longitude' from datarame;
    
    R = 6373.0  # approximate radius of earth in km    
    lat1, lon1 = from_coord
    lat2, lon2 = to_coord

    dlat = radians(lat2-lat1)
    dlon = radians(lon2-lon1)
    
    ## Calculate the distance between two coordinates in kilometers using the Haversine formula
    a = sin(dlat/2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * (sin(dlon/2) ** 2 )
    coord_distance = 2 * R * atan2(math.sqrt(a), sqrt(1-a))
    
    return coord_distance

In [20]:
def calculate_distance_with_zones(from_zone, to_zone):
    ##Calculate the distance between two taxi zones in kilometers using the centroid of each zone
    
    from_zone_coord = lookup_coords_for_taxi_zone_id(from_zone)
    to_zone_coord = lookup_coords_for_taxi_zone_id(to_zone)   # get two coordinates from two zone ids
    
    return calculate_distance_with_coords(from_zone_coord, to_zone_coord)

In [23]:
def add_distance_column(dataframe):
    ## Add a column to the DataFrame that contains the distance in kilometers between the pickup and dropoff locations
    
    if 'pickup_latitude' in dataframe.columns and 'pickup_longitude' in dataframe.columns and 'dropoff_latitude' in dataframe.columns and 'dropoff_longitude' in dataframe.columns:
        pickup_coords = list(zip(dataframe.pickup_latitude, dataframe.pickup_longitude))
        dropoff_coords = list(zip(dataframe.dropoff_latitude, dataframe.dropoff_longitude))

        distances = [calculate_distance_with_coords(from_coord, to_coord) for from_coord, to_coord in zip(pickup_coords, dropoff_coords)]

        dataframe['distance_km'] = distances
    elif 'PULocationID' in dataframe.columns and 'DOLocationID' in dataframe.columns:
        taxi_zones = gpd.read_file(TAXI_ZONES_SHAPEFILE)

        pickup_zones = taxi_zones[taxi_zones.LocationID.isin(dataframe.PULocationID)]
        dropoff_zones = taxi_zones[taxi_zones.LocationID.isin(dataframe.DOLocationID)]

        distances = [calculate_distance_with_zones(from_zone, to_zone) for from_zone, to_zone in zip(pickup_zones.itertuples(), dropoff_zones.itertuples())]

        dataframe['distance_km'] = distances
    else:
        raise ValueError('Dataframe does not contain the necessary columns.')

### Process Taxi Data

In [9]:
def get_all_urls_from_taxi_page(taxi_page):
    raise NotImplementedError()

In [10]:
def filter_taxi_parquet_urls(all_urls):
    raise NotImplementedError()

In [11]:
def get_and_clean_month(url):
    raise NotImplementedError()

In [12]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

In [13]:
def get_taxi_data():
    all_urls = get_all_urls_from_taxi_page(TAXI_URL)
    all_parquet_urls = find_taxi_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [14]:
taxi_data = get_taxi_data()

NotImplementedError: 

In [ ]:
taxi_data.head()

### Processing Uber Data

In [ ]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    df = df.dropna(how='any')      #remove missing values
    df = df.drop(columns=['Unnamed: 0','key','passenger_count'])      #remove unnecessary columns
    df = df.loc[df.pickup_longitude.between(-74.242330,-73.717047) & df.dropoff_longitude.between(-74.242330,-73.717047) 
         & df.pickup_latitude.between(40.560445,40.908524)& df.dropoff_latitude.between(40.560445,40.908524)]
          # remove locations out of range
    df.reset_index(drop=True, inplace=True)    #reset index
    return df

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    uber_dataframe = add_distance_column(uber_dataframe)
    uber_dataframe["pick_date"] = pd.to_datetime(uber_dataframe['pickup_datetime'])
    uber_dataframe["year"] = uber_dataframe["pick_date"].dt.year
    uber_dataframe["month"] = uber_dataframe["pick_date"].dt.month
    uber_dataframe["day"] = uber_dataframe["pick_date"].dt.day
    uber_dataframe["hour"] = uber_dataframe["pick_date"].dt.hour
    uber_dataframe["dayofweek"] = uber_dataframe["pick_date"].dt.dayofweek
    return uber_dataframe

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory):
    csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]
    return csv_files

In [ ]:
get_all_weather_csvs(WEATHER_CSV_DIR)

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_daily(csv_file):
    raise NotImplementedError()

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
daily_weather_data.head()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)